# Carregamento do dataset

### Informações gravadas em: https://github.com/bneurd/opt020

In [1]:
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import LabelEncoder
import mne

# Inicialização da lista para armazenar os dados de todos os participantes
all_data = []

# Loop externo para cada participante (1 a 5)
for participant in range(1, 6):
    participant_data = []  # Lista para armazenar os dados concatenados do participante

    # Loop interno para os dois arquivos de cada participante (_1 e _2)
    for session in range(1, 3):
        # Construir o caminho do arquivo
        file_path = f"../multi/Sub{participant}_{session}_multitarget.mat"
        #print(f"Loading file: {file_path}")

        try:
            # Carregar o arquivo
            f = loadmat(file_path)

            # Extrair os dados e labels
            data = f['Data'][0][0][7]  # Dados dos trials
            labels = f['Data'][0][0][8][0]  # Rótulos dos trials

            #print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")

            # Adicionar os dados à lista do participante
            participant_data.append(data)

        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    # Concatenar as duas sessões do participante ao longo da dimensão horizontal
    if participant_data:
        try:
            participant_data = np.concatenate(participant_data, axis=1)  # Concatenar ao longo de axis=1
            print(f"Participant {participant} concatenated data shape: {participant_data.shape}")
            all_data.append(participant_data)
        except Exception as e:
            print(f"Error concatenating data for participant {participant}: {e}")

# Verificar se all_data não está vazio antes de empilhar
if all_data:
    try:
        all_data = np.stack(all_data)  # Cria um array com dimensão (5, 8192, 20)
        print("Final shape:", all_data.shape)
    except Exception as e:
        print(f"Error stacking data: {e}")
else:
    print("No data to stack.")

Participant 1 concatenated data shape: (8192, 20)
Participant 2 concatenated data shape: (8192, 20)
Participant 3 concatenated data shape: (8192, 20)
Participant 4 concatenated data shape: (8192, 20)
Participant 5 concatenated data shape: (8192, 20)
Final shape: (5, 8192, 20)


Dado o exemplo acima, faça o carregamento dos cinco participantes, contatenando suas seções ou *trials*. Basta em um `for` externo aumentar a dimensionalidade para a quantidade de participantes e em um `for` interno, concatenar as seções com dez (10) *targets* cada uma.

Ao final, teremos um *shape* igual a `(5, 8192, 20)`.

### Utilizando a biblioteca MNE

Nesta etapa, vamos criar um objeto MNE para armazenar os dados juntamente com os labels.

Para este tipo de tarefa, é recomendo o uso do objeto Epochs, pois armazena tanto os dados quanto os metadados.

In [2]:
# inicialmente criamos o objeto info
import mne

n_channels = 1
sfreq = 512
ch_names = ['Oz']
ch_types = ['eeg']
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
info

<Info | 7 non-empty values
 bads: []
 ch_names: Oz
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: unspecified
 nchan: 1
 projs: []
 sfreq: 512.0 Hz
>

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import mne

# Shape dos dados: (5, 8192, 1) (5 participantes, 8192 amostras, 1 canal - Oz)
all_data = np.stack(all_data)  


data_correct = np.transpose(all_data, (0, 2, 1)) 



# Transformação dos rótulos
labels = np.array(['6.0', '7.0', '7.5', '8.2', '9.3'])  # Exemplo de rótulos para 5 participantes
le = LabelEncoder()
event_labels = le.fit_transform(labels)  # Transformação dos rótulos para valores numéricos

# Criação dos eventos
events = np.column_stack((
    np.array(range(len(labels))),  # Índices dos eventos (assumindo que os rótulos são sequenciais)
    np.zeros(len(labels), dtype=int),  # A coluna de 'zeros' para marcação do evento
    event_labels  # Rótulos transformados em valores numéricos
))

# Organizar o event_dict com os rótulos únicos
event_dict = {str(label): idx for idx, label in enumerate(le.classes_)}


# Verifique se o número de canais em 'info' e 'data_correct' coincide
assert len(info['ch_names']) == data_correct.shape[1], "O número de canais em 'info' deve ser igual ao número de canais em 'data_correct'"
# Criar o objeto EpochsArray com a forma correta de dados
mne_data = mne.EpochsArray(data_correct, info, events, event_id=event_dict)
# Salvar os dados
mne_data.save('ssvep-epo.fif', overwrite=True)


Data shape: (5, 20, 8192)
Events shape: (5, 3)
Unique labels: ['6.0' '7.0' '7.5' '8.2' '9.3']
Event dict: {'6.0': 0, '7.0': 1, '7.5': 2, '8.2': 3, '9.3': 4}
Not setting metadata
5 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Overwriting existing file.
